In [31]:
# from pydantic import BaseModel, Field
# from datetime import date

# class CreateTodoInput(BaseModel):
#     todo_name: str = Field(..., description="Name of the todo")
#     todo_checkbox: bool = Field(..., description="Whether it is completed")
#     todo_duedate: str = Field(..., description="Due date in YYYY-MM-DD format")


In [9]:
import os
import google.generativeai as genai #type:ignore
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

'''Returns an instance of Gemini LLM'''
def InitializeGeminiLLM():
    GeminiApiKey=os.getenv('GOOGLE_API_KEY')
    genai.configure(api_key=GeminiApiKey)
    GeminiLLM = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    return GeminiLLM

LLM=InitializeGeminiLLM()

In [10]:
from pydantic import Field,BaseModel
from datetime import date
from typing import Optional,List


class TodoBase(BaseModel):
    todo_name:str=Field(...,min_length=2,max_length=30,description="Name of the todo")
    todo_checkbox:bool=Field(False,description="Status of the todo")
    todo_duedate:date=Field(...,default_factory=date.today,description="Due date of the todo")

class TodoCreate(TodoBase):
    pass

In [11]:
request_message="""You are an AI productivity assistant, your job is to create pythin objects using the following details
{details}"""


In [12]:
from langchain.prompts.chat import HumanMessagePromptTemplate,ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

def ObjectCreator(details:str):
    parser=PydanticOutputParser(pydantic_object=TodoCreate)
    human_prompt=HumanMessagePromptTemplate.from_template("{request}/n{format_instructions}")
    chat_prompt=ChatPromptTemplate.from_messages([human_prompt])

    request_base="""You are an AI productivity assistant, your job is to create pythin objects using the following details\n\n"""
    request_message=request_base+details

    request=chat_prompt.format_prompt(
        request=request_message,
        format_instructions=parser.get_format_instructions()
    ).to_messages()

    results=LLM(request)
    result_values=parser.parse(results.content)

    return result_values



In [14]:
from langchain.agents import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
import firebase_admin #type:ignore
from firebase_admin import credentials,firestore #type:ignore
import uuid
import datetime
import json
def initialize_firestore():
    if not firebase_admin._apps:
        cred = credentials.Certificate(r"C:\Users\Shreyas\Desktop\OrbitOS\firebase_key.json")
        firebase_admin.initialize_app(cred)
    db = firestore.client()
    return db

db=initialize_firestore()



def create_todo_base(todo_data:TodoCreate):
   
    try:
        
        unique_id = f"todo{uuid.uuid4().hex[:8]}"
        todo_dict = {
            "todo_name": todo_data.todo_name,
            "todo_checkbox": todo_data.todo_checkbox,
            "todo_duedate": todo_data.todo_duedate.isoformat(),
            "unique_id": unique_id,
        }

        todo_name= todo_dict['todo_name']
        todo_date=todo_dict['todo_duedate']

        db.collection("Todos").document(unique_id).set(todo_dict)
        result= "Todo created successfully with following details:\n"+"todo name = "+todo_name+"\ntodo duedate = "+todo_date
        

        return result


    except Exception as e:
        return {"status": "error", "message": str(e)}
    

@tool
def create_todo_agent(todo_details:str):
          
     """
      Creates a new todo in Firestore.

    This function accepts three seperate parameters and not a single parameter.

    The parameters for this function are:
    - todo_name: name of the todo (str value)
    - todo_checkbox: whether it is completed (boolean value)
    - todo_duedate: due date in YYYY-MM-DD format (str value)

      """
     Todo=ObjectCreator(todo_details)
     return create_todo_base(Todo)

     

    
    
     






{'status': 'success', 'id': 'todo11afdd7b'}

In [15]:
from langchain_core.prompts import PromptTemplate
from langchain.agents import create_tool_calling_agent
prompt = hub.pull("hwchase17/react")

tools=[create_todo_agent]
agent = create_react_agent(LLM,tools,prompt)

c:\Users\Shreyas\Desktop\OrbitOS\venv\Lib\site-packages\langsmith\client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [17]:
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
human_input="Bro I want to learn drawing, my friend said to first learn drawing shapes, then perspective then anatomy. create me todos for three of these stages na pleasee"
agent_executor.invoke({"input":human_input})



> Entering new AgentExecutor chain...
Okay, I need to create three todos for learning drawing: "Learn Shapes", "Learn Perspective", and "Learn Anatomy". I will use the `create_todo_agent` tool for each. I will set the `todo_checkbox` to `False` for all of them, as they are new todos. I will set the `todo_duedate` to be one week apart from each other, starting from today. Today is 2024-01-26.

Thought: I need to create the first todo: Learn Shapes
Action: create_todo_agent
Action Input: {'todo_name': 'Learn Shapes', 'todo_checkbox': False, 'todo_duedate': '2024-02-02'}Todo created successfully with following details:
todo name = Learn Shapes
todo duedate = 2024-02-02Okay, I need to create the second todo: Learn Perspective
Action: create_todo_agent
Action Input: {'todo_name': 'Learn Perspective', 'todo_checkbox': False, 'todo_duedate': '2024-02-09'}Todo created successfully with following details:
todo name = Learn Perspective
todo duedate = 2024-02-09Okay, I need to create the third 

{'input': 'Bro I want to learn drawing, my friend said to first learn drawing shapes, then perspective then anatomy. create me todos for three of these stages na pleasee',
 'output': 'I have created three todos for you: Learn Shapes (due 2024-02-02), Learn Perspective (due 2024-02-09), and Learn Anatomy (due 2024-02-16).'}